# **UIEFT**

## **Essencial**

### **Bibliotecas e funcoes**

In [ ]:
import cv2

from scipy import ndimage
from scipy.ndimage import gaussian_filter

from PIL import Image, ImageOps

from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from skimage import io, color, filters, measure

import math
import numpy as np

import os
import sys
from glob import glob
from os.path import join
from ntpath import basename

### **Pipeline**

In [ ]:
def colorCorrection(imagem, intensidade):
  resultados = [] #vetor para receber os resultados das trasnformações
  rgb = cv2.split(imagem) #acesso a cada canal de cor
  saturacao = rgb[0].shape[0] * rgb[0].shape[1] * intensidade / 500.0 #200
  for canal in rgb:
      histograma = cv2.calcHist([canal], [0], None, [256], (0,256), accumulate=False)
      #low value
      lowvalue = np.searchsorted(np.cumsum(histograma), saturacao) #soma acumulada dos elementos valor inferior do histograma e encontra índices onde os elementos devem ser inseridos p/ ordenar
      #high value
      highvalue = 255-np.searchsorted(np.cumsum(histograma[::-1]), saturacao)#soma acumulada e sort valores superiores
      #tomar toda a informação (min/max) da curva linear para aplicar e gerar uma LUT de 256 valores a aplicar nos canais stretching
      lut = np.array([0 if i < lowvalue else (255 if i > highvalue else round(float(i-lowvalue)/float(highvalue-lowvalue)*255)) for i in np.arange(0, 256)], dtype="uint8")
      #mescla os canais de volta
      resultados.append(cv2.LUT(canal, lut))
  return cv2.merge(resultados)

In [ ]:
def gammaCorrection(image, gamma):  #0.7
#construir uma tabela de novos valores mapeados de pixel [0, 255] para seus valores gama ajustados
  invGamma = 1.0 / gamma
  table = np.array([((i / 255.0) ** invGamma) * 255
  for i in np.arange(0, 256)]).astype("uint8")
  #aplicar a correção
  return cv2.LUT(image, table)

In [ ]:
def mascaraNitidez(imagem, sigma, intensidade, kernel=(5, 5), threshold=0): #sigma 1.0/ 0.5, intensidade 2.0/ 1.0
  suavizacao = cv2.GaussianBlur(imagem, kernel, sigma)
  nitidez = float(intensidade + 1) * imagem - float(intensidade) * suavizacao
  nitidez = np.maximum(nitidez, np.zeros(nitidez.shape))
  nitidez = np.minimum(nitidez, 255 * np.ones(nitidez.shape))
  nitidez = nitidez.round().astype(np.uint8)
  if threshold > 0:
      contraste_baixo = np.absolute(imagem - suavizacao) < threshold
      np.copyto(nitidez, imagem, where=contraste_baixo)
  return nitidez

In [ ]:
def CLAHE(imagem):
  clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(4, 4))
  for i in range(3):
    imagem[:, :, i] = clahe.apply((imagem[:, :, i]))
  return imagem

In [ ]:
def add(a ,b, alfa, beta):
  fusao = cv2.addWeighted(a, alfa, b, beta, 0) #combina duas imagens
  #0.5/0.5 valores alfa e beta
  return fusao

def enhacement(imagem, brightness, contrast): #b=5 c=10
  img = np.int16(imagem)
  img = img * (contrast/127+1) - contrast + brightness
  img = np.clip(img, 0, 255)
  final = np.uint8(img)
  return final

### **Main**

In [ ]:
def UWE(image, cci, gamma, alfa, beta, sigma=0.5, intensidade=1.0, brightness=5, contrast=10):

  colorCorrected = colorCorrection(image, cci)
#   cv2_imshow(colorCorrected)
  gammaCorrected = gammaCorrection(colorCorrected, gamma)
#   cv2_imshow(gammaCorrected)
  edgeEnhacement = mascaraNitidez(gammaCorrected, sigma, intensidade)
#   cv2_imshow(edgeEnhacement)
  clahe = CLAHE(colorCorrected)
#   cv2_imshow(clahe)
  gammaCorrected2 = gammaCorrection(colorCorrected, gamma)
#   cv2_imshow(gammaCorrected2)
  edgeEnhacement2 = mascaraNitidez(gammaCorrected2, sigma, intensidade)
#   cv2_imshow(edgeEnhacement2)
  fusao = add(edgeEnhacement, edgeEnhacement2, alfa, beta)
#   cv2_imshow(fusao)

  output = enhacement(fusao, brightness, contrast)

  return output

### **Evaluation**

In [ ]:
def getUCIQE(img):
    img_BGR = img
    img_LAB = cv2.cvtColor(img_BGR, cv2.COLOR_BGR2LAB)
    img_LAB = np.array(img_LAB,dtype=np.float64)
    # Trained coefficients are c1=0.4680, c2=0.2745, c3=0.2576 according to paper.
    coe_Metric = [0.4680, 0.2745, 0.2576]

    img_lum = img_LAB[:,:,0]/255.0
    img_a = img_LAB[:,:,1]/255.0
    img_b = img_LAB[:,:,2]/255.0

    # item-1
    chroma = np.sqrt(np.square(img_a)+np.square(img_b))
    sigma_c = np.std(chroma)

    # item-2
    img_lum = img_lum.flatten()
    sorted_index = np.argsort(img_lum)
    top_index = sorted_index[int(len(img_lum)*0.99)]
    bottom_index = sorted_index[int(len(img_lum)*0.01)]
    con_lum = img_lum[top_index] - img_lum[bottom_index]

    # item-3
    chroma = chroma.flatten()
    sat = np.divide(chroma, img_lum, out=np.zeros_like(chroma, dtype=np.float64), where=img_lum!=0)
    avg_sat = np.mean(sat)

    uciqe = sigma_c*coe_Metric[0] + con_lum*coe_Metric[1] + avg_sat*coe_Metric[2]
    return uciqe

def getPSNR(im_true,im_test):
    im_true = np.resize(256, 256)
    im_test = np.resize(256, 256)
    return peak_signal_noise_ratio(im_true,im_test)

def SSIMM(im_true,im_test):
    im_true = np.resize(256, 256)
    im_test= np.resize(256, 256)
    return structural_similarity(im_true,im_test,  multichannel=True)



def mu_a(x, alpha_L=0.1, alpha_R=0.1):

    # Calculates the asymetric alpha-trimmed
    # Média aparada (dispersão)

    # sort pixels by intensity - for clipping
    x = sorted(x)
    # get number of pixels
    K = len(x)
    # calculate T alpha L and T alpha R
    T_a_L = math.ceil(alpha_L * K)
    T_a_R = math.floor(alpha_R * K)
    # calculate mu_alpha weight
    weight = (1 / (K - T_a_L - T_a_R))
    # loop through flattened image starting at T_a_L+1 and ending at K-T_a_R
    s = int(T_a_L + 1)
    e = int(K - T_a_R)
    val = sum(x[s:e])
    val = weight * val
    return val


def s_a(x, mu):
    val = 0
    for pixel in x:
        val += math.pow((pixel - mu), 2)
    return val / len(x)

def sobel(x):
    dx = ndimage.sobel(x, 0)
    dy = ndimage.sobel(x, 1)
    mag = np.hypot(dx, dy)
    mag *= 255.0 / np.max(mag)
    return mag


def eme(x, window_size):
      #Enhancement measure estimation
      #x.shape[0] = height
      #x.shape[1] = width

    # if 4 blocks, then 2x2...etc.
    k1 = x.shape[1] // window_size
    k2 = x.shape[0] // window_size

    # weight
    w = 2. / (k1 * k2)

    blocksize_x = window_size
    blocksize_y = window_size

    # make sure image is divisible by window_size - doesn't matter if we cut out some pixels
    x = x[:blocksize_y * k2, :blocksize_x * k1]

    val = 0
    for l in range(k1):
        for k in range(k2):
            block = x[k * window_size:window_size * (k + 1), l * window_size:window_size * (l + 1)]
            max_ = np.max(block)
            min_ = np.min(block)

            # bound checks, can't do log(0)
            if min_ == 0.0:
                val += 0
            elif max_ == 0.0:
                val += 0
            else:
                val += math.log(max_ / min_)
    return w * val


###########################################
#logAMEE
###########################################
def plip_g(x, mu=1026.0):
    return mu - x


def plip_theta(g1, g2, k):
    g1 = plip_g(g1)
    g2 = plip_g(g2)
    return k * ((g1 - g2) / (k - g2))


def plip_cross(g1, g2, gamma):
    g1 = plip_g(g1)
    g2 = plip_g(g2)
    return g1 + g2 - ((g1 * g2) / (gamma))


def plip_diag(c, g, gamma):
    g = plip_g(g)
    return gamma - (gamma * math.pow((1 - (g / gamma)), c))


def plip_multiplication(g1, g2):
    return plip_phiInverse(plip_phi(g1) * plip_phi(g2))
    # return plip_phiInverse(plip_phi(plip_g(g1)) * plip_phi(plip_g(g2)))


def plip_phiInverse(g):
    plip_lambda = 1026.0
    plip_beta = 1.0
    return plip_lambda * (1 - math.pow(math.exp(-g / plip_lambda), 1 / plip_beta));


def plip_phi(g):
    plip_lambda = 1026.0
    plip_beta = 1.0
    return -plip_lambda * math.pow(math.log(1 - g / plip_lambda), plip_beta)


def _uicm(x):
    R = x[:, :, 0].flatten()
    G = x[:, :, 1].flatten()
    B = x[:, :, 2].flatten()
    RG = R - G
    YB = ((R + G) / 2) - B
    mu_a_RG = mu_a(RG)
    mu_a_YB = mu_a(YB)
    s_a_RG = s_a(RG, mu_a_RG)
    s_a_YB = s_a(YB, mu_a_YB)
    l = math.sqrt((math.pow(mu_a_RG, 2) + math.pow(mu_a_YB, 2)))
    r = math.sqrt(s_a_RG + s_a_YB)
    return (-0.0268 * l) + (0.1586 * r)


def _uism(x):
    # get image channels
    R = x[:, :, 0]
    G = x[:, :, 1]
    B = x[:, :, 2]

    # first apply Sobel edge detector to each RGB component
    Rs = sobel(R)
    Gs = sobel(G)
    Bs = sobel(B)

    # multiply the edges detected for each channel by the channel itself
    R_edge_map = np.multiply(Rs, R)
    G_edge_map = np.multiply(Gs, G)
    B_edge_map = np.multiply(Bs, B)

    # get eme for each channel
    r_eme = eme(R_edge_map, 8)
    g_eme = eme(G_edge_map, 8)
    b_eme = eme(B_edge_map, 8)

    # coefficients
    lambda_r = 0.299
    lambda_g = 0.587
    lambda_b = 0.144

    return (lambda_r * r_eme) + (lambda_g * g_eme) + (lambda_b * b_eme)


def _uiconm(x, window_size):
    plip_lambda = 1026.0
    plip_gamma = 1026.0
    plip_beta = 1.0
    plip_mu = 1026.0
    plip_k = 1026.0

    # if 4 blocks, then 2x2...etc.
    k1 = x.shape[1] // window_size
    k2 = x.shape[0] // window_size

    # weight
    w = -1. / (k1 * k2)

    blocksize_x = window_size
    blocksize_y = window_size

    # make sure image is divisible by window_size - doesn't matter if we cut out some pixels
    x = x[:blocksize_y * k2, :blocksize_x * k1]

    # entropy scale - higher helps with randomness
    alpha = 1

    val = 0
    for l in range(k1):
        for k in range(k2):
            block = x[k * window_size:window_size * (k + 1), l * window_size:window_size * (l + 1), :]
            max_ = np.max(block)
            min_ = np.min(block)

            top = max_ - min_
            bot = max_ + min_

            if math.isnan(top) or math.isnan(bot) or bot == 0.0 or top == 0.0:
                val += 0.0
            else:
                val += alpha * math.pow((top / bot), alpha) * math.log(top / bot)

            # try: val += plip_multiplication((top/bot),math.log(top/bot))
    return w * val

def getUIQM(x):
    """
      Function to return UIQM to be called from other programs
      x: image
    """
    x = x.astype(np.float32)
    ### from https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7300447
    # c1 = 0.4680; c2 = 0.2745; c3 = 0.2576
    ### from https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7300447
    """
    c1 = 0.0282
    c2 = 0.2953
    c3 = 3.5753

    """
    #"""
    c1 = 0.4680
    c2 = 0.2745
    c3 = 0.2576
    #"""
    uicm = _uicm(x)
    print(f"UICM: {uicm}")
    uism = _uism(x)
    print(f"UISM: {uism}")
    uiconm = _uiconm(x, 8)
    print(f"UICONM: {uiconm}")
    uiqm = (c1 * uicm) + (c2 * uism) + (c3 * uiconm)
    return uiqm


### **Collect data to txt**

In [ ]:
def escreve(arquivo, vetor):
  with open(arquivo, 'w') as writefile:
    for h in range(len(vetor)):
      writefile.write(str(vetor[h]) + '\n')

### **Resize func**

In [ ]:
def resize(src, scale_percent):
    width = int(src.shape[1] * scale_percent / 100)
    height = int(src.shape[0] * scale_percent / 100)
    dsize = (width, height)
    output = cv2.resize(src, dsize)
    return output

### **Best results func**

In [ ]:
#Função que retorna a melhor variação de parâmetros a partir do resultado das métricas de qualidade de im. subaquáticas

def maior_valor(lista):
  if(len(lista) > 0):

    maior_valor = [lista[0][5], lista[0][6]]
    maior_indice = 0

    for i in range(0, len(lista)):
      if((lista[i][5] > maior_valor[0]) and (lista[i][6] > maior_valor[1])):
        maior_valor[0] = lista[i][5]
        maior_valor[1] = lista[i][6]
        maior_indice = i
    return maior_valor, maior_indice
  return -1, -1

## **Google**

In [ ]:
from google.colab import files
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
# plt.imshow(imagem, interpolation='nearest')
# plt.show()

## **Testes Datasets**

### **U45**

In [ ]:
DIR = '/content/drive/MyDrive/UWE/Data/U45_DATASET/U45 Dataset/U45/U45/' #diretório das imagens
DIR_results = '/content/drive/MyDrive/UWE/Data/U45_DATASET/U_45_results/'
arquivo = '/content/drive/MyDrive/UWE/Data/U45_DATASET/u45_results.txt'
melhores_resultados = [] #vetor que receberá os resultados

#percorrer pasta de arquivos
for filename in sorted(os.listdir(DIR)):
  resultados=[] #lista que recebe a string contendo os dados (imagem, parâmetros e resultados qualidade)
  caminho_img = f'{DIR}/{filename}'

  img = cv2.imread(caminho_img)
#   imgem = cv2.imread(caminho_img)
#   img = resize(imgem, 50)

  print(filename)

  #parâmetros que serão testados
  cci = [0.8, 1.0] #intensidade de correcao de cor
  gamma = [0.8, 0.9] #intensidade do gamma
  alfa=[0.4, 0.5, 0.6, 0.7] #intensidade do alfa da combinação
  beta=[0.4, 0.5, 0.6, 0.7] #intensidade do beta da combinação


  for i in range(len(cci)):
    for j in range(len(gamma)):
      for m in range(len(alfa)):
        for n in range(len(beta)):

          #image, cci, gamma, alfa, beta, sigma=0.5, intensidade=1.0, brightness=5, contrast=10
          restored = UWE(img, cci[i], gamma[j], alfa[m], beta[n])

          #metricas de qualidade
          uci = getUCIQE(restored)
          uiq = getUIQM(restored)

          #psnr = peak_signal_noise_ratio(img, restored)
          #ssim = structural_similarity(img, restored,  multichannel=True)

          string = str(filename), str(cci[i]), str(gamma[j]), str(alfa[m]), str(beta[n]), "{:.4f}".format(uci), "{:.4f}".format(uiq)
          resultados.append(string)

  maior, indice = maior_valor(resultados)

  if(maior != -1 and indice != -1):
    melhores_resultados.append(resultados[indice])

  print(f'maior {maior}, indice {indice}, vetor[indice] {resultados[indice]}')

  my_str = '_'.join(resultados[indice]) .replace('0', '').replace('.', '').replace('png', "") + '.png'
  print(my_str)
  cv2.imwrite(f'{DIR_results}{str(my_str)}', img)
escreve(arquivo, melhores_resultados)

In [ ]:
import pandas as pd
# Read the text file
with open('/content/drive/MyDrive/UWE/Data/U45_DATASET/u45_results.txt', 'r') as f:
    lines = f.readlines()

# Split each line into a list of strings
data = [line.strip().replace('(', '').replace(')', '').split(', ') for line in lines]

# Create a pandas DataFrame with the data
df = pd.DataFrame(data, columns=['Filename', 'CCI', 'Gamma', 'Alpha', 'Beta', 'UIQM', 'UCIQE'])
df[['CCI', 'Gamma', 'Alpha', 'Beta', 'UIQM', 'UCIQE']] = df[['CCI', 'Gamma', 'Alpha', 'Beta', 'UIQM', 'UCIQE']].apply(lambda x: x.str.strip("'"))
# Convert the non-string columns to float
df[['CCI', 'Gamma', 'Alpha', 'Beta', 'UIQM', 'UCIQE']] = df[['CCI', 'Gamma', 'Alpha', 'Beta', 'UIQM', 'UCIQE']].astype(float)

# Print the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/UWE/Data/U45_DATASET/u45results.csv', index=False)


In [ ]:
avg_uiqm = df['UIQM'].mean()
avg_uciqe = df['UCIQE'].mean()

# Create a new DataFrame with the averages
df_avg = pd.DataFrame({'UCIQE': [avg_uciqe], 'UIQM': [avg_uiqm]}, index=['Average'])

# Print the DataFrame
print(df_avg)
df.to_csv('/content/drive/MyDrive/UWE/Data/U45_DATASET/u45_results_av.csv', index=False)


### **UIEB**

In [ ]:
DIR = '/content/drive/MyDrive/UWE/Data/tudo/' #diretório das imagens
DIR_results = '/content/drive/MyDrive/UWE/Data/all/'
arquivo = '/content/drive/MyDrive/UWE/Data/all_results.txt'
melhores_resultados = [] #vetor que receberá os resultados

#percorrer pasta de arquivos
for filename in sorted(os.listdir(DIR)):
  resultados=[] #lista que recebe a string contendo os dados (imagem, parâmetros e resultados qualidade)
  caminho_img = f'{DIR}/{filename}'

#   img = cv2.imread(caminho_img)
  imgem = cv2.imread(caminho_img)
  img = resize(imgem, 35)

  print(filename)

  #parâmetros que serão testados
  cci = [0.7, 0.8, 0.9, 1.0] #intensidade de correcao de cor
  gamma = [0.6, 0.7, 0.8, 0.9] #intensidade do gamma
  alfa=[0.4, 0.5, 0.6, 0.7] #intensidade do alfa da combinação
  beta=[0.4, 0.5, 0.6, 0.7] #intensidade do beta da combinação


  for i in range(len(cci)):
    for j in range(len(gamma)):
      for m in range(len(alfa)):
        for n in range(len(beta)):

          #image, cci, gamma, alfa, beta, sigma=0.5, intensidade=1.0, brightness=5, contrast=10
          restored = UWE(img, cci[i], gamma[j], alfa[m], beta[n])

          #metricas de qualidade
          uci = getUCIQE(restored)
          uiq = getUIQM(restored)

          #psnr = peak_signal_noise_ratio(img, restored)
          #ssim = structural_similarity(img, restored,  multichannel=True)

          string = str(filename), str(cci[i]), str(gamma[j]), str(alfa[m]), str(beta[n]), "{:.4f}".format(uci), "{:.4f}".format(uiq)
          resultados.append(string)

  maior, indice = maior_valor(resultados)

  if(maior != -1 and indice != -1):
    melhores_resultados.append(resultados[indice])

#   print(f'maior {maior}, indice {indice}, vetor[indice] {resultados[indice]}')
  print(f'{resultados[indice]}')

  my_str = '_'.join(resultados[indice]) .replace('0', '').replace('.', '').replace('png', "") + '.png'
  print(my_str)
  cv2.imwrite(f'{DIR_results}{str(my_str)}', img)
escreve(arquivo, melhores_resultados)


In [ ]:
with open('/content/drive/MyDrive/UWE/Data/UIEB_DATASET/UIEB Dataset/uieb_resultados.txt', 'r') as f:
    lines = f.readlines()

# Split each line into a list of strings
data = [line.strip().replace('(', '').replace(')', '').split(', ') for line in lines]

# Create a pandas DataFrame with the data
df = pd.DataFrame(data, columns=['Filename', 'CCI', 'Gamma', 'Alpha', 'Beta', 'UIQM', 'UCIQE'])
df[['CCI', 'Gamma', 'Alpha', 'Beta', 'UIQM', 'UCIQE']] = df[['CCI', 'Gamma', 'Alpha', 'Beta', 'UIQM', 'UCIQE']].apply(lambda x: x.str.strip("'"))
# Convert the non-string columns to float
df[['CCI', 'Gamma', 'Alpha', 'Beta', 'UIQM', 'UCIQE']] = df[['CCI', 'Gamma', 'Alpha', 'Beta', 'UIQM', 'UCIQE']].astype(float)

# Print the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('/content/drive/MyDrive/UWE/Data/UIEB_DATASET/UIEB Dataset/uieb_resultados.csv', index=False)


In [ ]:
avg_uiqm = df['UIQM'].mean()
avg_uciqe = df['UCIQE'].mean()

# Create a new DataFrame with the averages
df_avg = pd.DataFrame({'UCIQE': [avg_uciqe], 'UIQM': [avg_uiqm]}, index=['Average'])

# Print the DataFrame
print(df_avg)
df.to_csv('/content/drive/MyDrive/UWE/Data/UIEB_DATASET/UIEB Dataset/uieb_resultados_av.csv', index=False)


### **CARU**

In [ ]:
DIR = '/content/drive/MyDrive/UWE/Data/U45_DATASET/U45 Dataset/U45/U45' #diretório das imagens
arquivo = '/content/drive/MyDrive/UWE/Data/u45_melhores.txt' #arquivo texto onde serão registrados os resultados

melhores_resultados = [] #vetor que receberá os resultados

#percorrer pasta de arquivos
for filename in os.listdir(DIR):

  resultados=[] #lista que recebe a string contendo os dados (imagem, parâmetros e resultados qualidade)

  caminho_img = f'{DIR}/{filename}'

  img = cv2.imread(caminho_img)

  #parâmetros que serão testados
  cci = [0.8, 1.0] #intensidade de correcao de cor
  gamma = [0.6, 0.7] #intensidade do gamma
  alfa=[0.4, 0.5, 0.6, 0.7] #intensidade do alfa da combinação
  beta=[0.4, 0.5, 0.6, 0.7] #intensidade do beta da combinação


  for i in range(len(cci)):
    for j in range(len(gamma)):
      for m in range(len(alfa)):
        for n in range(len(beta)):

            #image, cci, gamma, alfa, beta, sigma=0.5, intensidade=1.0, brightness=5, contrast=10
          restored = UWE(img, cci[i], gamma[j], alfa[m], beta[n])

          #metricas de qualidade
          uci = getUCIQE(restored)
          uiq = getUIQM(restored)

          #psnr = peak_signal_noise_ratio(img, restored)
          #ssim = structural_similarity(img, restored,  multichannel=True)

          #se resultados forem superiores ao minimo esperado
          if uiq>=4.0 and uci>=0.8:
            string = str(filename), str(cci[i]), str(gamma[j]), str(alfa[m]), str(beta[n]), "{:.4f}".format(uci), "{:.4f}".format(uiq)
            resultados.append(string)


  maior, indice = maior_valor(resultados)
  melhores_resultados.append(resultados[indice])
  #print(f'maior {maior}, indice {indice}, vetor[indice] {resultados[indice]}')

#print(melhores_resultados)
#[print(melhor) for melhor in melhores_resultados]


print(*melhores_resultados, sep = "\n")

escreve(arquivo, melhores_resultados)